In [1]:
from pyspark.sql import SparkSession, DataFrame

spark = SparkSession.builder.getOrCreate()

In [56]:
# """Change cwd to root of project so we can use usual filepaths."""

# from os import chdir, getcwd

# for frag in reversed(getcwd().split('/')):
#     if frag != 'metadatadriven':
#         chdir('..')
#     else:
#         print(getcwd())
#         break

In [34]:
"""Read json from dbfs or local."""

from json import load
from os import path


def abs_to_rel(filepath: str) -> str:
    """Convert `/home` to `home`."""
    return ''.join(filepath[1:]) if filepath[0] == '/' else filepath


def local_file_api_path(filepath: str) -> str:
    """Convert `/home` to `dbfs/home` if exists."""
    dbfsfilepath = path.join('dbfs', abs_to_rel(filepath))
    return dbfsfilepath if path.exists(dbfsfilepath) else filepath


def load_json(filepath: str) -> dict:
    """Load json local file api."""
    with open(local_file_api_path(filepath), 'r') as f:
        return load(f)


meta = load_json(path.join('../mnt/houses', 'meta.json'))
glom(meta, 'input.options', default={})

{'header': True}

In [50]:
folder = '../mnt/houses'
path.join(folder, '**/*.csv')

'../mnt/houses/**/*.csv'

In [111]:
from glom import glom

def read(metajsonpath: str) -> DataFrame:
    """Read folder containing metadata."""
    meta = load_json(metajsonpath)
    format = glom(meta, 'input.format', default='text')
    fields = glom(meta, 'fields', default=[])
    path = glom(meta, 'input.path')
    schema = None
    return (
        spark
        .read
        .load(
            path=path,
            format=format,
            schema=schema,
            **glom(meta, 'input.options', default={})
        )
    )


df_houses = read('../mnt/houses.json')
df_houses.show()

df_friends = read('../mnt/friends.json')
df_friends.show()

+---+----------+
| id|      city|
+---+----------+
|  5|Lutjebroek|
|  4|     Zwaag|
|  0| Enkhuizen|
|  2| Enkhuizen|
|  3| Enkhuizen|
|  1| Venhuizen|
+---+----------+

+---------+---+--------+
|firstname| id|lastname|
+---------+---+--------+
|   Stefan|  0|  Schenk|
|    Chris|  1| Wijnker|
|      Tom|  2|  Kamper|
|     Dion|  3| Buysman|
|     Dave|  4|  Conijn|
|  Dimitri|  5|  Stathi|
|     Mark|  6|Breukink|
+---------+---+--------+

